<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/New_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import Required LIbraries and Mount Google Drive

In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from google.colab import drive
from datetime import datetime
import calendar

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#2. Load & Preprocess Data

In [34]:
input_file_path = "/content/drive/MyDrive/VERGER/Revenue_Forecasting/forecasting_data.csv"
df = pd.read_csv(input_file_path)

# Clean and ensure correct dtypes
df = df.dropna(subset=['Year', 'Month_No'])
df['Year'] = df['Year'].astype(int)
df['Month_No'] = df['Month_No'].astype(int)
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce').fillna(0)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0)

# Apply log1p transform for stability
df['Revenue'] = np.log1p(df['Revenue'])
df['Quantity'] = np.log1p(df['Quantity'])

# Create date column and sort
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month_No'].astype(str) + '-01')
df = df.sort_values(['Account','Product','Type','Date']).reset_index(drop=True)

# Encode categorical features
label_encoders = {}
for col in ['Account','Product','Type']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str)) # Ensure fitting on string type
    label_encoders[col] = le

#3. Feature Engineering

In [35]:
# lag features
def create_lags(group, lags=[1,2,3,12]):
    group = group.sort_values('Date')
    for lag in lags:
        group[f'Revenue_lag{lag}'] = group['Revenue'].shift(lag).fillna(0)
        group[f'Quantity_lag{lag}'] = group['Quantity'].shift(lag).fillna(0)
    return group

# rolling averages
def add_rolling_features(group):
    group = group.sort_values('Date')
    group['Revenue_MA3'] = group['Revenue'].rolling(3).mean().fillna(0)
    group['Revenue_MA6'] = group['Revenue'].rolling(6).mean().fillna(0)
    group['Quantity_MA3'] = group['Quantity'].rolling(3).mean().fillna(0)
    group['Quantity_MA6'] = group['Quantity'].rolling(6).mean().fillna(0)
    return group

df = df.groupby(['Account','Product','Type']).apply(create_lags).reset_index(drop=True)
df = df.groupby(['Account','Product','Type']).apply(add_rolling_features).reset_index(drop=True)

# Drop first 12 rows per group to remove lag/rolling artifacts
df = df.groupby(['Account','Product','Type']).apply(lambda x: x.iloc[12:]).reset_index(drop=True)

# Group-level statistics
df['Account_mean_rev'] = df.groupby('Account')['Revenue'].transform('mean')
df['Product_mean_rev'] = df.groupby('Product')['Revenue'].transform('mean')
df['Type_mean_rev'] = df.groupby('Type')['Revenue'].transform('mean')
df['Account_mean_qty'] = df.groupby('Account')['Quantity'].transform('mean')
df['Product_mean_qty'] = df.groupby('Product')['Quantity'].transform('mean')
df['Type_mean_qty'] = df.groupby('Type')['Quantity'].transform('mean')

# Cyclical features
df['Month_Sin'] = np.sin(2*np.pi*df['Month_No']/12)
df['Month_Cos'] = np.cos(2*np.pi*df['Month_No']/12)
df['Year_Since'] = df['Year'] - df['Year'].min()
df['Quarter_No'] = df['Quarter'].str.replace("Q","").astype(int)

/tmp/ipython-input-2741454320.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['Account','Product','Type']).apply(create_lags).reset_index(drop=True)
/tmp/ipython-input-2741454320.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['Account','Product','Type']).apply(add_rolling_features).reset_index(drop=True)
/tmp/ipython-input-2741454320.py:22: DeprecationWarning: Da

#4. Features and Targets Columns

In [36]:
# Feature Columns
feature_cols = [
    'Account','Product','Type',
    'Month_No','Month_Sin','Month_Cos','Year_Since','Quarter_No',
    'Revenue_lag1','Revenue_lag2','Revenue_lag3','Revenue_lag12',
    'Quantity_lag1','Quantity_lag2','Quantity_lag3','Quantity_lag12',
    'Revenue_MA3','Revenue_MA6','Quantity_MA3','Quantity_MA6',
    'Account_mean_rev','Product_mean_rev','Type_mean_rev',
    'Account_mean_qty','Product_mean_qty','Type_mean_qty'
]

# Target Variables
X = df[feature_cols]
y_rev = df['Revenue']
y_qty = df['Quantity']

#5.Train/Test Split

In [37]:
X_train, X_test, y_train_rev, y_test_rev, y_train_qty, y_test_qty = train_test_split(
    X, y_rev, y_qty, test_size=0.2, random_state=42
)

#6. Hyperparameter Grid

In [38]:
param_grid = {
    'n_estimators': [500, 800, 1000, 1200],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [4, 6, 8, 10],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0.5, 1, 1.5]
}

#7. Randomized Search Setup

In [39]:
def tune_xgb_model(X_train, y_train):
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    random_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_grid,
        n_iter=50,
        scoring='neg_mean_squared_error',
        cv=3,
        verbose=2,
        random_state=42,
        n_jobs=-1
    )
    random_search.fit(X_train, y_train)
    print("✅ Best parameters found:", random_search.best_params_)
    return random_search.best_estimator_

#8. Train Revenue Model

In [40]:
model_rev = tune_xgb_model(X_train, y_train_rev)
model_rev.fit(X_train, y_train_rev, eval_set=[(X_test, y_test_rev)], verbose=False)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
✅ Best parameters found: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 800, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.03, 'gamma': 0.3, 'colsample_bytree': 0.8}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.3, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.03, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=800,
             n_jobs=None, num_parallel_tree=None, ...)

#9. Train Quantity Model

In [41]:
model_qty = tune_xgb_model(X_train, y_train_qty)
model_qty.fit(X_train, y_train_qty, eval_set=[(X_test, y_test_qty)], verbose=False)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
✅ Best parameters found: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 800, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.03, 'gamma': 0.3, 'colsample_bytree': 0.8}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.3, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.03, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=800,
             n_jobs=None, num_parallel_tree=None, ...)

#10. Model Evaluation

In [42]:
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)
    y_pred_inv = np.expm1(y_pred)
    y_test_inv = np.expm1(y_test)
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    print(f"{target_name} -> RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")

evaluate_model(model_rev, X_test, y_test_rev, 'Revenue')
evaluate_model(model_qty, X_test, y_test_qty, 'Quantity')

Revenue -> RMSE: 18459.17, MAE: 6097.81, R²: 0.8179
Quantity -> RMSE: 197.34, MAE: 115.97, R²: 0.5515


#11. Forecasting Function

In [43]:
def forecast(account, product, typ, year, month):
    acc = label_encoders['Account'].transform([account])[0]
    prod = label_encoders['Product'].transform([product])[0]
    tp = label_encoders['Type'].transform([typ])[0]

    subset = df[(df['Account']==acc) & (df['Product']==prod) & (df['Type']==tp)]
    if subset.empty:
        return {'Product': product, 'Type': typ, 'Forecast_Quantity': 1, 'Forecast_Revenue': "1"}

    last_row = subset.iloc[-1]

    feature_row = {
        'Account': acc, 'Product': prod, 'Type': tp,
        'Month_No': month,
        'Month_Sin': np.sin(2*np.pi*month/12),
        'Month_Cos': np.cos(2*np.pi*month/12),
        'Year_Since': year - df['Year'].min(),
        'Quarter_No': ((month-1)//3)+1,
        'Revenue_lag1': last_row['Revenue'],
        'Revenue_lag2': last_row['Revenue_lag1'],
        'Revenue_lag3': last_row['Revenue_lag2'],
        'Revenue_lag12': last_row['Revenue_lag12'],
        'Quantity_lag1': last_row['Quantity'],
        'Quantity_lag2': last_row['Quantity_lag1'],
        'Quantity_lag3': last_row['Quantity_lag2'],
        'Quantity_lag12': last_row['Quantity_lag12'],
        'Revenue_MA3': last_row['Revenue_MA3'],
        'Revenue_MA6': last_row['Revenue_MA6'],
        'Quantity_MA3': last_row['Quantity_MA3'],
        'Quantity_MA6': last_row['Quantity_MA6'],
        'Account_mean_rev': last_row['Account_mean_rev'],
        'Product_mean_rev': last_row['Product_mean_rev'],
        'Type_mean_rev': last_row['Type_mean_rev'],
        'Account_mean_qty': last_row['Account_mean_qty'],
        'Product_mean_qty': last_row['Product_mean_qty'],
        'Type_mean_qty': last_row['Type_mean_qty']
    }

    X_new = pd.DataFrame([feature_row])[feature_cols]
    qty_pred = np.expm1(model_qty.predict(X_new)[0])
    rev_pred = np.expm1(model_rev.predict(X_new)[0])

    qty_pred = max(1, int(round(qty_pred)))
    rev_pred = max(1, int(round(rev_pred)))

    return {'Product': product, 'Type': typ, 'Forecast_Quantity': qty_pred, 'Forecast_Revenue': f"{rev_pred:,}"}

#12. Forecast Orders Function

In [44]:
order_counts = df.groupby(['Account','Product','Type']).size().reset_index(name='order_count')

def forecast_orders(account_name, start_year, start_month, end_year, end_month):
    acc = label_encoders['Account'].transform([account_name])[0]
    products_types = df[df['Account']==acc][['Product','Type']].drop_duplicates()
    forecast_rows = []

    month_range = pd.date_range(
        start=datetime(start_year, start_month, 1),
        end=datetime(end_year, end_month, 1),
        freq='MS'
    )

    for single_date in month_range:
        year = single_date.year
        month = single_date.month
        for _, row in products_types.iterrows():
            prod = label_encoders['Product'].inverse_transform([row['Product']])[0]
            typ = label_encoders['Type'].inverse_transform([row['Type']])[0]

            pred = forecast(account_name, prod, typ, year, month)

            oc = order_counts[
                (order_counts['Account']==acc) &
                (order_counts['Product']==row['Product']) &
                (order_counts['Type']==row['Type'])
            ]['order_count']

            num_orders = int(oc.values[0]) if len(oc) > 0 else 1
            per_order_qty = max(1, int(round(pred['Forecast_Quantity'] / num_orders)))
            per_order_rev = max(1, int(round(int(pred['Forecast_Revenue'].replace(',','')) / num_orders)))

            for i in range(num_orders):
                forecast_rows.append({
                    'Year': year,
                    'Month': f"{month:02d}",
                    'Account': account_name,
                    'Product': pred['Product'],
                    'Type': pred['Type'],
                    'Forecast_Quantity': per_order_qty,
                    'Forecast_Revenue': f"{per_order_rev:,}"
                })

    return pd.DataFrame(forecast_rows)

#13. Run the Forecasting

In [45]:
result_df = forecast_orders('RBUS', 2025, 8, 2025, 12)
print(result_df)

    Year Month Account            Product          Type  Forecast_Quantity  \
0   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
1   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
2   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
3   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
4   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
5   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
6   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
7   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
8   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
9   2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
10  2025    08    RBUS  CINNAMON BARK OIL  Conventional                 64   
11  2025    08    RBUS  CINNAMON BARK OIL  Conventional         